In [37]:
import numpy as np

In [2]:
from utils.data import cv_prepare_data, train_test_split
from utils.cv import *
from utils.pass_quality import approx_pq

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
Using Theano backend.
/home/novikov/Documents/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


# cross validation

In [20]:
n_experiments = 10

results = []

for i in range(n_experiments):
    print 'loading data...'
    data = cv_prepare_data()
    
    print 'splitting_data...'
    X_train, X_test, y_train, y_test, y_train_lab, y_test_lab = train_test_split(data, test_size=0.2, n_prev=15)
    
    print 'building a model...'
    model = cv_build_model(X_train)
    
    
    print 'fitting the model...'
    model = cv_fit_model(model, X_train, y_train, nb_epoch=20, weights=True)
    
    print 'choosing threshold...'
    threshold = cv_threshold(X_train, y_train, model)
    
    print 'validation...'
    y_train_pred = cv_predict(model, X_train, y_train, threshold)
    y_test_pred = cv_predict(model, X_test, y_test, threshold)

    pq_test = approx_pq(y_test, y_test_pred)
    pq_train = approx_pq(y_train, y_train_pred)
    
    results.append(
        {'exp_id' : i,
         'data' : data,
         'y_train' : y_train, 
         'y_test' : y_test, 
         'y_train_lab' : y_train_lab,
         'y_test_lab' : y_test_lab,
         'y_train_pred' : y_train_pred,
         'y_test_pred' : y_test_pred,
         'threshold' : threshold,
         'pq_test' : pq_test,
         'pq_train' : pq_train,
         'model' : model,
        }
    )
    clear_output(wait=True)

choosing threshold...
validation...


In [41]:
np.mean([res['pq_test'][0] for res in results]), np.std([res['pq_test'][0] for res in results])

(0.88541291646570852, 0.036887644906313563)

In [42]:
np.mean([res['pq_train'][0] for res in results]), np.std([res['pq_train'][0] for res in results])

(0.88723492595422027, 0.0090353364060408257)